<b>Updated 5th November 2018<br/></b>
Author: Phillip Crout (pc494) <br/> 
Comment: To reflect the move to diffpy

<center><h4> ALERT: This should be downloaded and run, rather than viewed directly on github </h4></center>

In [ ]:
%matplotlib tk
import numpy as np
import hyperspy.api as hs
import pyxem as pxm
import diffpy.structure
from matplotlib import pyplot as plt
from pyxem.generators.indexation_generator import IndexationGenerator
from pyxem.utils.sim_utils import peaks_from_best_template
from pyxem.utils.plot import generate_marker_inputs_from_peaks
from pyxem.libraries.structure_library import StructureLibrary

<h2><center> Creating a "fake" dataset </h2></center>

First we set up two crystal structures

In [ ]:
latt = diffpy.structure.lattice.Lattice(5, 5, 5, 90, 90, 90)
atom = diffpy.structure.atom.Atom(atype='Si', xyz=[0, 0, 0], lattice=latt)
si = diffpy.structure.Structure(atoms=[atom], lattice=latt)

In [ ]:
latt = diffpy.structure.lattice.Lattice(3, 3, 5, 90, 90, 120)
atom = diffpy.structure.atom.Atom(atype='Ga', xyz=[0, 0, 0], lattice=latt)
ga = diffpy.structure.Structure(atoms=[atom], lattice=latt)

And some simulation paramaters

In [ ]:
size = 256 #pattern size in pixels
radius=1.2 #reciprocal radius
ediff = pxm.DiffractionGenerator(300., 0.025) #eV and relrod size

We now create 4 seperate patterns, 2 for each crystal, one at 0 degress and one at 10 degrees, but this is all very artificial

In [ ]:
sample_lib = StructureLibrary(['si','ga'],[si,ga],[[(10,0,0),(0,0,0)],[(0,0,0),(10,0,0)]])
sample_lib.struct_lib

diff_gen = pxm.DiffractionLibraryGenerator(ediff)
library = diff_gen.get_diffraction_library(sample_lib,
                                           calibration=1 / 64,
                                           reciprocal_radius=0.8,
                                           half_shape=(64,64),
                                           with_direct_beam=False)

data_silicon = []
data_gallium = []

for theta in [0,10]:
    _ = library.get_library_entry(phase='si',angle=(theta,0,0))['Sim'].as_signal(128,0.03,1)
    data_silicon.append(_)
    _ = library.get_library_entry(phase='ga',angle=(theta,0,0))['Sim'].as_signal(128,0.03,1)
    data_gallium.append(_)
        
data = [x.data for x in data_silicon] + [x.data for x in data_gallium]

test_data = pxm.ElectronDiffraction(np.asarray(data).reshape(2,2,128,128))
test_data.set_diffraction_calibration(1/64)

In [ ]:
test_data.plot()

test_data now contains the 4 patterns we will attempt to match to, we move onto creating a library

<center><h2> Doing the mapping </h2></center>

First we need to create a list of potential rotations

In [ ]:
rot_list = []
for theta in np.linspace(0,15,50): 
    rot_list.append((theta, 0, 0.))    #needs to be a tuple

And then a structure library (see the docstrings for details)

In [ ]:
search_lib = StructureLibrary(['si','ga'],[si,ga],[rot_list,rot_list])

The next step generates a library, which contains simulated diffraction data

In [ ]:
diff_gen = pxm.DiffractionLibraryGenerator(ediff)
library = diff_gen.get_diffraction_library(search_lib,
                                            calibration=1/64,
                                            reciprocal_radius=0.7,
                                            half_shape=(64,64),
                                            with_direct_beam=True)

which we then correlate with the test (normally experimental) dataset

In [ ]:
indexer = IndexationGenerator(test_data, library)
phase=["si","ga"] 
match_results = indexer.correlate(n_largest=2,keys=phase)

we then have a range of ways of working with this output, but here we simply plot it to show that the method has worked as anticipated

In [ ]:
peaks= match_results.map(peaks_from_best_template,phase=phase,library=library,inplace=False)
mmx,mmy = generate_marker_inputs_from_peaks(peaks)
test_data.plot(cmap='viridis') 
for mx,my in zip(mmx,mmy):
    m = hs.markers.point(x=mx,y=my,color='red',marker='x')
    test_data.add_marker(m,plot_marker=True,permanent=True)

Remeber our rotations are anti-clockwise!

<h2><center> Appendix </center></h2>

Below is playground to help you get used to our rotation convention. "f1" and "f2" will be plotted as figure 1 and 2 respectively (assuming no figures are open when you run the cell) - and each is set to an euler triplet

In [ ]:
f1 = (90,-3,-90)
f2 = (90,+3,-90)
sample_lib = StructureLibrary(['si'],[si],[[f1,f2]])
diff_gen = pxm.DiffractionLibraryGenerator(ediff)
library = diff_gen.get_diffraction_library(sample_lib,
                                           calibration=1 / 64,
                                           reciprocal_radius=0.8,
                                           half_shape=(64,64),
                                           with_direct_beam=False)

_ = library.get_library_entry(phase='si',angle=f1)['Sim'].as_signal(128,0.03,1)
plt.figure()
plt.imshow(_,cmap='viridis',vmax=0.1)

_ = library.get_library_entry(phase='si',angle=f2)['Sim'].as_signal(128,0.03,1)
plt.figure()
plt.imshow(_,cmap='viridis',vmax=0.1)